# Docker

Позволяет обеспечить изоляцию приложений в Linux-системах (используются механизмы ядра - *cgroups* и *namespaces*).  Помогает автоматизировать процесс развертывания приложений и запуска в изолированном окружении.

## Работа с образами
```console
docker build -t image_name .
```

Загрузка готового образа с DockerHub
```console
docker pull tensorflow/tensorflow:2.2.0-py3-jupyter
```

Список имеющихся образов
```console
docker images
```

Удалить образ
```console
docker rmi image_name
```

## Запуск контейнера
```console
docker run --name container_name image_name  
```

Используем параметры точки входа (чаще всего команда, если точка входа - `bash`). Ключ `-it` позволяет работать в терминале
```console
docker run --name container_name -it image_name cmd
```

Удаление данных контейнера после завершения работы приложения
```console
docker run --name container_name -it --rm image_name bash
```

Проброс портов
```console
docker run --name container_name -p 8000:8000 image_name
```

Отображаем директорию из host-системы на директорию в контейнере
```console
docker run --name container_name -v `pwd`/app:/app image_name
```

# Работа с контейнером 
Список работающих контейнеров
```bash
docker ps -a
```

Например:
```console
user@ubuntu:~$ docker ps 

CONTAINER ID        IMAGE               COMMAND                 CREATED             STATUS              PORTS               NAMES
f0cc91079770        telegram            "/home/user/Telegram"   3 days ago          Up 3 days                               telegram
 
```
Список работающий и остановленных контейнеров
```console
docker ps -a
```

Остановить контейнер
```console
docker stop container_name 
```

Убить (`SIGKILL`) контейнер или послать сигнал
```bash
docker kill container_name 
```

Запустить остановленный контейнер
```bash
docker start container_name 
```

Скопировать `path1` из хоста в `path2`в контейнере. Можно наоборот
```bash
docker cp path1 container_name:path2
```

Показать логи контейнера
```bash
docker logs container_name
```

Создать новый образ из контейнера
```bash
docker commit container_name image_name
```

Запустить команду в контексте контейнера
```bash
docker exec -it container_name cmd
```

Низкоуровневая информация о контейнере
```bash
docker inspect container_name
```

# Слои

<img src="https://docs.docker.com/storage/storagedriver/images/overlay_constructs.webp"/>

```console
docker history image_name
```

Например:
```console
user@ubuntu:~$ docker history telegram

IMAGE               CREATED             CREATED BY                                      SIZE                COMMENT
1705d31f665a        3 days ago          /bin/sh -c #(nop)  ENTRYPOINT ["/home/user/T…   0B                  
88d2adb91b00        3 days ago          /bin/sh -c #(nop)  ENV QT_XKB_CONFIG_ROOT=/u…   0B                  
9f63a912ac16        3 days ago          /bin/sh -c #(nop)  USER user                    0B                  
42f7157c794d        3 days ago          /bin/sh -c #(nop) WORKDIR /home/user            0B                  
3582309f7267        3 days ago          /bin/sh -c mkdir -p $HOME/.local/share/Teleg…   105MB               
9e9a1c174b55        3 days ago          /bin/sh -c #(nop) COPY dir:42a4b167c29fdf9b6…   105MB               
972ee88b0c86        2 months ago        /bin/sh -c apt-get update && apt-get -y upgr…   163MB               
31e67ad20612        2 months ago        /bin/sh -c useradd --create-home --home-dir …   398kB               
3d17fb568438        2 months ago        /bin/sh -c #(nop)  ENV HOME=/home/user          0B                  
6e36b5fdb873        2 months ago        /bin/sh -c #(nop)  LABEL maintainer=Christop…   0B                  
cd6d8154f1e1        13 months ago       /bin/sh -c #(nop)  CMD ["/bin/bash"]            0B                  
<missing>           13 months ago       /bin/sh -c mkdir -p /run/systemd && echo 'do…   7B                  
<missing>           13 months ago       /bin/sh -c sed -i 's/^#\s*\(deb.*universe\)$…   2.76kB              
<missing>           13 months ago       /bin/sh -c rm -rf /var/lib/apt/lists/*          0B                  
<missing>           13 months ago       /bin/sh -c set -xe   && echo '#!/bin/sh' > /…   745B                
<missing>           13 months ago       /bin/sh -c #(nop) ADD file:3df374a69ce696c21…   84.1MB  
```

# Создание образов

1. Порядок операций в `Dockerfile` имеет значение для кеширования
2. Лучше объединять связанные команды в одну

```bash
RUN apt update && apt install -y \
    apt-utils \
    software-properties-common \
    xz-utils \
    wget \
    --no-install-recommends && \
    apt clean && \
    rm -rf /var/lib/apt/lists/* /tmp/* /var/tmp/*
```

3. Сборка должна быть воспроизводимой, лучше чтобы она не зависела от каких-то внешних факторов, как окружение хоста. 

# Песочницы

### HDFS, Hadoop, Hive, 

1. Устанавливаем `Docker` и запускаем терминал

2. Скачиваем контейнер с bigtop sandbox
    ```console 
    docker pull bigtop/sandbox:1.2.1-ubuntu-16.04-hdfs_yarn_hive_pig        
    ```

3. Запускаем, предварительно рекомендуется войти в директорию, куда склонирован git-репозиторий курса:

    ```console
    docker run -d --hostname=quickstart --name=hadoop-sandbox --privileged=true \
        -p 50070:50070 -p 14000:14000 -p 8088:8088 -p 19888:19888 \
        -p 9083:9083 -p 10000:10000 -p 10002:10002  \
        -v `pwd`:/course \
        bigtop/sandbox:1.2.1-ubuntu-16.04-hdfs_yarn_hive_pig
    ```

4. Ждем некоторое время, когда контейнер запустится, затем в терминале

    ```console
    docker exec -it hadoop-sandbox bash
    ```    
6. В терминале

    ```console
    apt update -y & apt install -y python3 python3-pip hadoop-httpfs  
    service hadoop-httpfs start 
    python3 -m pip install mrjob 
    ```   
7. Использование

    Контейнер можно остановить
    ```console
    docker stop  hadoop-sandbox
    ``` 

    и потом опять запустить
    ```console
    docker start  hadoop-sandbox
    ```

    Если потребуется пробросить дополнительный порт или раздел, то можно сделать так - создать образ из существующего контейнера, удалить контейнер и запустить новый, указав дополнительные параметры:


    ```console
    docker stop hadoop-sandbox
    docker commit hadoop-sandbox hadoop-image
    docker rm hadoop-sandbox
    docker run -d --hostname=quickstart --name=hadoop-sandbox --privileged=true \
        -p 50070:50070 -p 14000:14000 -p 8088:8088 -p 19888:19888 \
        -p 9083:9083 -p 10000:10000 -p 10002:10002  \
        -v `pwd`:/course \
        hadoop-image
    ```

    После запуска контейнера на хост-системе можно проверить доступность сервисов:

    - http://localhost:50070 - HDFS WEB UI
    - http://localhost:14000 - HttpFS REST API
    - http://localhost:8088 - Resource Manager
    - http://localhost:19888 - Hadoop History Server
    

### Spark

```console
curl -LO https://raw.githubusercontent.com/bitnami/containers/main/bitnami/spark/docker-compose.yml
docker-compose up
````